<a href="https://colab.research.google.com/github/Vyoma-Kanani/TensorFlow/blob/main/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# CIFAR10 Dataset
---

This dataset is contains natural images of 10 different classes like airplane, automobile, bird, etc.

Number of training images are 50000 <br>
Number of test images are 10000<br>
Each image is of 32*32 pixels<br>
RGB colors - 3 channels 



```
Concept of Convolution Neutal Network is used in this notebook.
```



In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import cifar10

Loading data

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# Converting into float32
# Normalization so that pixel vallues are between 0 and 1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0 

## Model

#### Sequential 

In [ ]:
model = keras.Sequential([
  # Here as we are using convolution neural network we are going to maintain the shape i.e 32 height 32 width and 3 channels for RGB
  keras.Input(shape = (32, 32, 3)),
  # Here the first argument is for how many channels we want this convolution layer to output
  # Writting single integer means same length and height i.e. (3,3)  
  # Padding are valid(default) and same. 
  # Same refers that the output will have same number of inputs which are passed in our case 32. Valid the output is going to change depending on kernal size. 
  layers.Conv2D(32, 3, padding = "valid", activation='relu' ),
  layers.MaxPooling2D(pool_size=(2,2)),  #MaxPooling pool_size is a default argument
  layers.Conv2D(64, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, 3, activation='relu'),
  # Output Layer of convolution network
  layers.Flatten(),
  # Intermediate Layer with 64 nodes fully connected
  layers.Dense(64, activation='relu'),
  # Output of the whole network with 10 nodes
  layers.Dense(10),
])

print(model.summary)

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x7f72121531d0>>


#### Functional API

In [ ]:
def my_model():
  inputs = keras.Input(shape=(32,32,3))
  #Here we are not using activation function as we want to sent it to batchNorm first then conv function and then to activation function
  x =layers.Conv2D(32,3)(inputs)    
  x = layers.BatchNormalization()(x)
  x= keras.activations.relu(x) 
  x = layers.MaxPooling2D()(x)
  x = layers.Conv2D(64, 5, padding='same')(x)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.Conv2D(128,3)(x)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.Flatten()(x)
  x = layers.Dense(64, activation='relu')(x)
  outputs = layers.Dense(10)(x)

  model = keras.Model(inputs = inputs, outputs = outputs)
  
  return model

In [ ]:
model = my_model()

Compiling Model

In [ ]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(lr=3e-4),
    metrics = ['accuracy'], 
)

model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size = 64, verbose= 2)


Epoch 1/10
782/782 - 242s - loss: 1.3176 - accuracy: 0.5302
Epoch 2/10
782/782 - 241s - loss: 0.9085 - accuracy: 0.6809
Epoch 3/10
782/782 - 240s - loss: 0.7442 - accuracy: 0.7401
Epoch 4/10
782/782 - 241s - loss: 0.6299 - accuracy: 0.7776
Epoch 5/10
782/782 - 243s - loss: 0.5312 - accuracy: 0.8153
Epoch 6/10
782/782 - 242s - loss: 0.4537 - accuracy: 0.8424
Epoch 7/10
782/782 - 242s - loss: 0.3836 - accuracy: 0.8664
Epoch 8/10
782/782 - 246s - loss: 0.3209 - accuracy: 0.8897
Epoch 9/10
782/782 - 247s - loss: 0.2578 - accuracy: 0.9132
Epoch 10/10
782/782 - 247s - loss: 0.2121 - accuracy: 0.9303
157/157 - 12s - loss: 1.1163 - accuracy: 0.6947


[1.1163208484649658, 0.6947000026702881]

So it seems that there is huge gap between Training Accuracy and Test Accuracy<br>
So the Model is overfitting the training data<br><br>
Method to reduce overfitting is call **Regularization**
<br> Here we are using 3 methods


*   L2
*   Dropout
* Batch Norm





### Modified Above Code

In [ ]:
def my_model():
    inputs = keras.Input(shape=(32, 32, 3))
    x = layers.Conv2D(32, 3, padding="same", kernel_regularizer=regularizers.l2(0.01),)(
        inputs
    )
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, padding="same", kernel_regularizer=regularizers.l2(0.01),)(
        x
    )
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(
        128, 3, padding="same", kernel_regularizer=regularizers.l2(0.01),
    )(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.01),)(
        x
    )
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(10)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


model = my_model()

In [ ]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(lr=3e-4),
    metrics = ['accuracy'], 
)

model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size = 64, verbose= 2)

Epoch 1/10
782/782 - 133s - loss: 2.7236 - accuracy: 0.3823
Epoch 2/10
782/782 - 132s - loss: 1.8824 - accuracy: 0.4787
Epoch 3/10
782/782 - 132s - loss: 1.6334 - accuracy: 0.5118
Epoch 4/10
782/782 - 133s - loss: 1.5224 - accuracy: 0.5378
Epoch 5/10
782/782 - 132s - loss: 1.4536 - accuracy: 0.5548
Epoch 6/10
782/782 - 134s - loss: 1.3968 - accuracy: 0.5699
Epoch 7/10
782/782 - 134s - loss: 1.3803 - accuracy: 0.5776
Epoch 8/10
782/782 - 134s - loss: 1.3442 - accuracy: 0.5861
Epoch 9/10
782/782 - 134s - loss: 1.3251 - accuracy: 0.5960
Epoch 10/10
782/782 - 133s - loss: 1.3133 - accuracy: 0.6008
157/157 - 7s - loss: 1.4044 - accuracy: 0.5850


[1.4043841361999512, 0.5849999785423279]